In [14]:
import pandas as pd

data_folder = 'data/'
itens_file_name = 'valor_unitario_aprovado_sample.csv'

itens_file_path = data_folder + itens_file_name

dt = pd.read_csv(itens_file_path)
dt.head()

,Unnamed: 0,Item,ValorUnitarioAprovado,Data,idPRONAC
0,1894129,Material de consumo,7000.00,2014-07-31 07:27:29,179966
1,2003098,Locação de teatro,465.39,2015-05-05 16:38:24,185736
2,925488,Banheiro químico,250.00,2012-03-16 00:00:00,143223
3,1326505,Técnico de som,3500.00,2013-01-07 00:00:00,155368
4,136947,Cartaz,300.00,2010-01-01 00:00:00,114412


### Cleaning data

**Eliminating items with approved value equal to zero (0)**

There are some items with approved value equal to zero (0), for example the item with index **154** in the above table. For now they have no use, so lets get rid of them.

In [15]:
dt[dt.ValorUnitarioAprovado != 0.0]
dt.head()

,Unnamed: 0,Item,ValorUnitarioAprovado,Data,idPRONAC
0,1894129,Material de consumo,7000.00,2014-07-31 07:27:29,179966
1,2003098,Locação de teatro,465.39,2015-05-05 16:38:24,185736
2,925488,Banheiro químico,250.00,2012-03-16 00:00:00,143223
3,1326505,Técnico de som,3500.00,2013-01-07 00:00:00,155368
4,136947,Cartaz,300.00,2010-01-01 00:00:00,114412


### Number of distinct items

In [16]:
print(len(dt['Item'].unique()))

1106


**Lots of distinct items**

There **1106** distinct items in that sample. It will be hard to plot them one by one, so it's a good idea to plot some sample of those items (the most frequent ones, for example).

**Getting the most frequent items**

In [17]:
top_frequent = dt['Item'].value_counts().head(10)
display(top_frequent)

Transporte Local / Locação de Automóvel / Combustível               354
Passagens Aéreas (Descrever os trechos na tela de deslocamentos)    326
Refeição                                                            325
Assessor de imprensa                                                324
Assistente de produção                                              291
Produtor Executivo                                                  279
Contador                                                            261
Hospedagem sem Alimentação                                          243
Cartaz                                                              232
Remuneração para captação de recursos                               225
Name: Item, dtype: int64